In [37]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_covtype
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,SimpleRNN,LSTM,GRU,Dropout
# 1. 데이터 
datasets = (fetch_covtype())

x = pd.DataFrame(datasets["data"])
y = (datasets["target"])

In [38]:
def split_x(dataset,timesteps) : 
    tmp = []
    for i in range(len(dataset)-timesteps + 1):
        subset = dataset[i : (i+ timesteps)]
        tmp.append(subset)
    return np.array(tmp)

x = split_x(x,5)
y = split_x(y,5)

print(x,"\n",y)

[[[2.596e+03 5.100e+01 3.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [2.590e+03 5.600e+01 2.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [2.804e+03 1.390e+02 9.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [2.785e+03 1.550e+02 1.800e+01 ... 0.000e+00 0.000e+00 0.000e+00]
  [2.595e+03 4.500e+01 2.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]]

 [[2.590e+03 5.600e+01 2.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [2.804e+03 1.390e+02 9.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [2.785e+03 1.550e+02 1.800e+01 ... 0.000e+00 0.000e+00 0.000e+00]
  [2.595e+03 4.500e+01 2.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [2.579e+03 1.320e+02 6.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]]

 [[2.804e+03 1.390e+02 9.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [2.785e+03 1.550e+02 1.800e+01 ... 0.000e+00 0.000e+00 0.000e+00]
  [2.595e+03 4.500e+01 2.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [2.579e+03 1.320e+02 6.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [2.606e+03 4.500e+01 7.000e+00 ... 0.000e+

In [39]:
print(x.shape,y.shape)

(581008, 5, 54) (581008, 5)


In [40]:
# 모델을 리쉐이프해서 7개 데이터 를 (3,1) 로 만들자
# x = x.reshape(len(x),4,4,1)
# print(x.shape)
# print(x)

In [41]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train,y_test = train_test_split(
    x,y, shuffle = True,
    train_size=0.7,
    
)
print(x.shape,y.shape)


(581008, 5, 54) (581008, 5)


In [42]:
# RNN 모델구성
model = Sequential([
    # input_shape 는 행 무시 
    # LSTM 은 3차원을 -> 2차원으로 변환함 flatten 필요없다
    # 따라서 LSTM 을 두번쓰면 오류발생
    LSTM(64,input_shape=(5,54)),
    Dropout(0.2),
    Dense(16,activation="relu"),
    Dense(1)
])
# 파라미터의 값 :
# RNN x 4 (출력 , 입력 ,삭제게이트)
# ( unit 개수 ) * (unit 개수 + feature 개수)*4
# 기본 rnn ㅇㅇ이처럼 파라미터의 값이 매우 많아서 연산량이 매우 많다
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                30464     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 16)                1040      
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 31,521
Trainable params: 31,521
Non-trainable params: 0
_________________________________________________________________


In [44]:
# 컴파일 훈련
model.compile(loss="mae",optimizer="adam")
model.fit(x_train,y_train,epochs=10,batch_size=128)

Epoch 1/10
3178/3178 [==============================] - 15s 4ms/step - loss: 0.7501
Epoch 2/10
3178/3178 [==============================] - 14s 4ms/step - loss: 0.7335
Epoch 3/10
3178/3178 [==============================] - 14s 4ms/step - loss: 0.7309
Epoch 4/10
3178/3178 [==============================] - 14s 5ms/step - loss: 0.7274
Epoch 5/10
3178/3178 [==============================] - 14s 4ms/step - loss: 0.7314
Epoch 6/10
3178/3178 [==============================] - 14s 4ms/step - loss: 0.7284
Epoch 7/10
3178/3178 [==============================] - 14s 4ms/step - loss: 0.7258
Epoch 8/10
3178/3178 [==============================] - 14s 4ms/step - loss: 0.7270
Epoch 9/10
3178/3178 [==============================] - 14s 4ms/step - loss: 0.7254
Epoch 10/10
3178/3178 [==============================] - 15s 5ms/step - loss: 0.7243


In [45]:
loss = model.evaluate(x_test,y_test)

print(loss)

5447/5447 [==============================] - 13s 2ms/step - loss: 0.7201
0.7201328277587891
